## Mlxtend を用いたアソシエーション分析

アソシエーション分析の機能を提供するパッケージとして mlxtend がある。<br>
https://anaconda.org/conda-forge/mlxtend

第12回のワインのデータについて mlxtend を用いてアソシエーション分析を行う。<br>
手順は以下の通り：
1. データを mlxtend が処理できる形態に変換
2. データに合わせた最小サポート、最小コンフィデンスの設定
3. 頻出アイテムセットの作成、相関ルールの生成

### インストール

以下のコマンドによりインストール：
```
conda install -c conda-forge mlxtend
```

In [ ]:
import pandas as pd
from pprint import pprint

# ItemIDとアイテム名の辞書
item_dict = {
    'I1': 'フランスワイン',
    'I2': 'イタリアワイン',
    'I3': 'スペインワイン',
    'I4': 'チリワイン',
    'I5': '国産ワイン'
}

# ワインの購買履歴のデータ
trans = [
    # TID: T1
    [1, 'I1'],
    [1, 'I2'],
    [1, 'I4'],
    [1, 'I5'],
    # TID: T2
    [2, 'I2'],
    [2, 'I3'],
    [2, 'I5'],
    # TID: T3
    [3, 'I1'],
    [3, 'I2'],
    [3, 'I4'],
    [3, 'I5'],
    # TID: T4
    [4, 'I1'],
    [4, 'I2'],
    [4, 'I3'],
    [4, 'I5'],
    # TID: T5
    [5, 'I1'],
    [5, 'I2'],
    [5, 'I3'],
    [5, 'I4'],
    [5, 'I5'],
    # TID: T6
    [6, 'I2'],
    [6, 'I3'],
    [6, 'I4']
]

# pandas DataFrame に変換
trans_df = pd.DataFrame(trans, columns=['TID', 'ItemID'])

In [ ]:
# mlxtend が処理できる形態に変換
# 1. TIDごとのリストを作成
trans_data = []

for tid in trans_df.TID.unique():
    # ItemID
    trans_data.append(trans_df.query('TID=={}'.format(tid)).ItemID.tolist())
    # アイテム名
    #trans_data.append([item_dict[i] for i in trans_df.query('TID=={}'.format(tid)).ItemID])

# 確認
print('TIDごとのリスト')
pprint(trans_data)

# 2. 作成したリストから横持ち形式（行がTID）のデータを作成
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_data = te.fit(trans_data).transform(trans_data)
df = pd.DataFrame(te_data, columns=te.columns_)

# 確認
df

In [ ]:
# 最小サポート: 0.5
min_sup = 0.5
# 最小コンフィデンス
min_confidence = 0.7

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Apriori アルゴリズムにより頻出アイテムセットを作成
frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)

# 相関ルールの生成
# - 最小コンフィデンスを指定する場合
#   - metric='confidence' 
#   - min_threshold: 最小コンフィデンスの値
_df_rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)
df_rules = _df_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
# 確認
df_rules
# antecedents (L), consequents (R)

In [ ]:
# lift値が1以上, confidenceが1のルールについて、liftで降順にソートして表示
df_rules.query('lift>=1 and confidence==1').sort_values('lift', ascending=False)